**1. Importing the librairies**

In [2]:
!pip install combo

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# standard imports and setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
# model evaluation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit, cross_validate, StratifiedKFold
from sklearn.metrics import *
# pipelines
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
# data preparation
from sklearn.preprocessing import *
# from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, RFECV
from sklearn.utils import resample
# from imblearn.datasets import make_imbalance
# from imblearn.over_sampling import SMOTE
# # machine learning
# from sklearn.linear_model import *
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC, LinearSVC
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier, XGBRegressor
#------------------
from __future__ import division
from __future__ import print_function
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from sklearn.metrics import roc_curve, roc_auc_score
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from time import time
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.metrics import plot_precision_recall_curve
from pyod.models.hbos import HBOS


In [4]:
!pip install pyod

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
 
import pandas as pd
dataset = pd.read_csv(r"C:\Users\NaumanRafique\Downloads\nourasaeed721-attachments\wustl.csv",usecols=['SIntPkt','Pulse_Rate',
 'dMaxPktSz','DstLoad', 'Trans','SrcLoad','sMinPktSz','DIntPktAct','DIntPkt','DstBytes','SpO2','DstGap','Loss','Label','DIA',
 'TotPkts','sMaxPktSz','Load','Rate','pSrcLoss','SrcBytes','DstJitter','Temp','SYS','pDstLoss','Heart_rate','dMinPktSz', 'SrcJitter','SIntPktAct','Resp_Rate','TotBytes','ST','pLoss','Dur','SrcGap'])

print(dataset.shape)
 

(16318, 35)


In [6]:
2+2

4

In [7]:
df_columns = ['Data Sample', 'Testing Size',  
              'ABOD', 'CBLOF', 'FB',
              'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM'
              ,'PCA'
             ]
 







random_state = 42




from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

import pandas as pd

# creating data frames for calculating 

df_time = pd.DataFrame(columns=df_columns)
df_precision = pd.DataFrame(columns=df_columns)
df_recall = pd.DataFrame(columns=df_columns)

# List of time,precision,recall 

list_time  =[]
list_precision  =[]
list_recall  =[]

dataset_sample = [1000,5000,10000,15000,16318]
testing_size= [0.2,0.3,0.4]

loc = 0

for sample in dataset_sample:
    
    dataset_sample = dataset.sample(sample)
    
    x = dataset_sample.iloc[:,:-1].values
    y = dataset_sample.iloc[:,-1].values
        
        
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
        
        
    classifierslist = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
    }

        
        
    
    for size in testing_size:
        
        
        
        x_train, x_test, y_train, y_test= train_test_split(x,y, test_size = size, random_state =0)
        
        x_train = sc.fit_transform(x_train)
        x_test = sc.fit_transform(x_test)  
        
      
        
        
        
        
        list_time.extend([sample,size])
        list_precision.extend([sample,size])
        list_recall.extend([sample,size])
        
        for clf_name, clf in classifierslist.items():
            
            t0 = time()
            clf.fit(x_train)
            test_scores = clf.decision_function(x_test)
            t1 = time()
            duration = round(t1 - t0, ndigits=4)
            
            roc = round(roc_auc_score(y_test, test_scores), ndigits=4)            
            prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
             
            
            list_time.append(duration)
            list_precision.append(prn)
            list_recall.append(roc)
        df_time.loc[loc] =list_time
        df_precision.loc[loc] =list_precision
        df_recall.loc[loc] =list_recall
        list_time  =[]
        list_precision  =[]
        list_recall  =[]
        loc+=1
        
            

            #accuracy = metrics.accuracy_score(y_test, y_pred)
#             y_test_pred = clf.predict(x_test)  # outlier labels (0 or 1)

#             print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
#                   'execution time: {duration}s'.format(clf_name=clf_name, roc=roc, prn=prn, duration=duration))

#             roc_list.append(roc)
#             prn_list.append(prn)

        






C:\Users\NaumanRafique\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\Users\NaumanRafique\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\Users\NaumanRafique\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\Users\NaumanRafique\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\Users\NaumanRafique\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix a

In [8]:
df_time

,Data Sample,Testing Size,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,1000.0,0.2,2.2606,1.9453,0.1829,2.156,0.3398,0.0570,0.0150,1.1532,0.1049,0.0189
1,1000.0,0.3,0.2459,0.0980,0.2449,0.022,0.3898,0.0760,0.0230,1.1464,0.0879,0.0070
2,1000.0,0.4,0.2488,0.0969,0.2569,0.022,0.3748,0.0969,0.0220,1.1573,0.0710,0.0054
3,5000.0,0.2,1.2123,0.1519,1.0984,0.034,0.7756,0.3778,0.1079,1.8373,2.6865,0.0245
4,5000.0,0.3,1.2813,0.1529,1.1154,0.033,0.6976,0.4597,0.1159,1.7489,2.3077,0.0209
5,5000.0,0.4,1.2983,0.1299,0.9535,0.032,0.6596,0.4797,0.0899,1.8711,1.8260,0.0200
6,10000.0,0.2,2.8903,0.2528,2.9723,0.049,1.6311,1.3333,0.3138,2.6062,12.3509,0.0567
7,10000.0,0.3,3.9807,0.3228,3.2911,0.053,1.4012,1.8429,0.3368,2.6823,10.7179,0.0377
8,10000.0,0.4,3.5780,0.4157,3.1702,0.042,1.3392,2.0208,0.7036,2.5285,8.2983,0.0525
9,15000.0,0.2,6.7971,0.3758,7.7655,0.066,2.5126,4.0047,0.9484,3.4583,31.6808,0.0611


In [9]:
df_recall

,Data Sample,Testing Size,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,1000.0,0.2,0.7201,0.6754,0.6709,0.6422,0.6576,0.6760,0.6592,0.6228,0.6651,0.6603
1,1000.0,0.3,0.7367,0.7087,0.6233,0.6871,0.6617,0.7074,0.5965,0.6427,0.6586,0.6645
2,1000.0,0.4,0.7126,0.7170,0.5961,0.6556,0.7115,0.7139,0.5267,0.6523,0.6537,0.6652
3,5000.0,0.2,0.5611,0.6360,0.4600,0.7221,0.6978,0.6670,0.4748,0.5987,0.6643,0.6714
4,5000.0,0.3,0.5812,0.6422,0.4847,0.7216,0.7110,0.6705,0.5007,0.6155,0.6604,0.6714
5,5000.0,0.4,0.5953,0.6294,0.4977,0.7294,0.7281,0.6803,0.5048,0.6082,0.6621,0.6705
6,10000.0,0.2,0.6098,0.5716,0.5167,0.6132,0.7095,0.6795,0.5136,0.6345,0.6668,0.6831
7,10000.0,0.3,0.5902,0.6098,0.5360,0.6944,0.7214,0.6918,0.5486,0.6232,0.6600,0.6719
8,10000.0,0.4,0.6053,0.6008,0.5336,0.6934,0.7243,0.6936,0.5413,0.6173,0.6661,0.6761
9,15000.0,0.2,0.6223,0.6806,0.5404,0.5910,0.6888,0.7025,0.5326,0.6331,0.6767,0.6813


In [10]:
df_precision

,Data Sample,Testing Size,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,1000.0,0.2,0.5238,0.5238,0.0476,0.5000,0.5238,0.5238,0.0000,0.5238,0.5238,0.5238
1,1000.0,0.3,0.4865,0.4865,0.2703,0.5135,0.5135,0.4865,0.0811,0.4865,0.4865,0.4865
2,1000.0,0.4,0.5000,0.4600,0.1000,0.2600,0.5400,0.5200,0.0600,0.5200,0.5200,0.5200
3,5000.0,0.2,0.1395,0.3023,0.1163,0.3203,0.4961,0.3953,0.1163,0.4574,0.3798,0.4109
4,5000.0,0.3,0.1479,0.2663,0.0769,0.2663,0.4556,0.3550,0.1006,0.4615,0.3550,0.4083
5,5000.0,0.4,0.1366,0.2247,0.0573,0.3216,0.4890,0.3568,0.0617,0.4581,0.3612,0.4229
6,10000.0,0.2,0.1236,0.1798,0.0824,0.2659,0.4944,0.3446,0.0487,0.4794,0.3371,0.4419
7,10000.0,0.3,0.1483,0.1867,0.1458,0.3606,0.5038,0.4399,0.1637,0.4680,0.3683,0.4297
8,10000.0,0.4,0.1673,0.1751,0.1187,0.3444,0.4942,0.4163,0.1128,0.4572,0.3638,0.4416
9,15000.0,0.2,0.1270,0.3892,0.0514,0.1946,0.4757,0.4730,0.0595,0.4757,0.4865,0.4892


In [11]:

df_time.to_csv("TimeData.csv")
df_precision.to_csv("PrecisionData.csv")
df_recall.to_csv("RecallData.csv")

Making confusion matrix and measurment the accur